In [5]:
import numpy as np
from functions import splitting, evaluate_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import time
import pickle

In [9]:
X_train, X_test, y_train, y_test = splitting('all')

# Assuming y_train and y_test are 2D arrays
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

     datetime       P    RH    U
0  2014010101  1008.3  0.05  9.2
1  2014010102  1008.8  0.00  9.3
2  2014010103  1008.9  0.00  9.1
3  2014010104  1009.1  0.00  9.3
4  2014010105  1009.0  0.00  9.1


/Users/sahilnakrani/Documents/weather forecast/src/Machine-Learning/Regression-Models/models/with_all_features/functions.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[['P', 'RH', 'U']] = scaler.fit_transform(X_train[['P', 'RH', 'U']])
/Users/sahilnakrani/Documents/weather forecast/src/Machine-Learning/Regression-Models/models/with_all_features/functions.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[['P', 'RH', 'U']] = scaler.transform(X_test[['P', 'RH', 'U']])


In [10]:
# Define the decision tree regression model
rf_model = RandomForestRegressor(max_features='sqrt', random_state=42)

# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [11]:
total_time_start = time.time()
# Perform grid search cross-validation
grid_search_rf = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Get the best hyperparameters
best_params_rf = grid_search_rf.best_params_

start_time = time.time()
# Initialize the decision tree regression model with the best hyperparameters
rf_model_best = RandomForestRegressor(**best_params_rf, random_state=42)
rf_model_best.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
total_time = end_time - total_time_start

# Save the trained model to a .pickle file
with open('/Users/sahilnakrani/Documents/weather forecast/src/Machine-Learning/Regression-Models/trained_models/RandomForestRegression/with_all_features/RandomForestRegressor.pkl', 'wb') as f:   
    pickle.dump(rf_model_best, f)

# Save training times to a text file
with open("/Users/sahilnakrani/Documents/weather forecast/src/Machine-Learning/Regression-Models/trained_models/RandomForestRegression/with_all_features/model_training_time.txt", "w") as f:
    f.write("RandomForestRegressor Training Time : {:.6f} seconds\n".format(training_time))
    f.write("Total Time with GridSearchCV : {:.6f} seconds\n".format(total_time))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [12]:
# Make predictions
y_pred_dt_best = rf_model_best.predict(X_test)

#evaluting the Model's performance.
evaluate_model(rf_model_best, X_test, y_test, 'RandomForestRegression', 'all')

Model: RandomForestRegression
Mean Squared Error: 31.911270404080494
Root Mean Squared Error: 5.6490061430379495
R2 Score: 0.37508662393647796
